In [1]:
import pandas as pd
import pickle
from myst_nb import glue

In [2]:
# Define variables with glue here
scores_df = pd.read_csv("../results/tables/model_comparison_results.csv")
glue("dt_test_score", scores_df['test_score'].values[1], display = False) 
glue("lr_test_score", scores_df['test_score'].values[2], display = False) 
glue("dummy_score", scores_df['test_score'].values[0], display = False) 
glue("model_score_table", scores_df, display=False)

coef_df = pd.read_csv("../results/tables/model_comparison_results.csv").round(2)
glue("coef_table", coef_df, display=False)

feature_importance_df = pd.read_csv("../results/tables/feature_importances.csv").round(3)
model_comparison_df = pd.read_csv("../results/tables/model_comparison_results.csv").round(3)
confusion_matrix_dt = pd.read_csv("../results/tables/confusion_matrix_DT.csv")
glue("highest_importance_coeff", feature_importance_df.iloc[4,2],display=False)
glue("optimized_knn_test_score", model_comparison_df.iloc[3,3],display=False)
glue("dt_test_score", model_comparison_df.iloc[1,3],display=False)
glue("false_negative", confusion_matrix_dt.iloc[1,1],display=False)
glue("false_positive", confusion_matrix_dt.iloc[0,2],display=False)

best_params = pd.read_csv("../results/tables/best_hyperparams.csv")
glue("knn_bestparam", best_params.loc[:, "n_neighbours"][0],display=False)
glue("tree_bestparam", best_params.loc[:, "max_depth"][0],display=False)

## Summary
This project endeavors to develop a predictive classification model for ascertaining an individual's diabetic status, while comparing the efficiency of optimized decision tree and optimized k-nearest neighbours (k-nn) algorithms. The dataset used in this analysis is collected through the Behavioral Risk Factor Surveillance System (BFRSS) by the Centers for Disease Control and Prevention (CDC) for the year 2015. Notably, the primary determinant influencing the prediction is identified as the feature Body Mass Index (BMI), displaying a coefficient of {glue:text}`highest_importance_coeff`  as revealed by the logistic regression model. The optimized decision tree model demonstrates a test score of {glue:text}`dt_test_score`,while the optimized k-nn model yields a test score of {glue:text}`optimized_knn_test_score`. Both of the test scores are relatively close to the validation score which shows that the model will generalized well to unseen data, however, there is still room for improvement in the test score. 

Our final classsifer is chosen to be the Desicion Tree model as it has both the highest test and train score in comparison with the optimized k-nn model. However, from the confusion matrix, we can see that our model has made {glue:text}`false_negative` type II error, which predicted the individual as non-diabetic when they are actually diabetic. On the other hand, it has also made {glue:text}`false_positive` type I error, which predicted the individual as diabetic when they are actually non-diabetic. In our case, both type II error and type I error is equivalently important as injecting insulin in a non-diabetic individual can be fatal and vice versa. Hence, we chose f1 score as our scoring matrix which is a hormonic balance between both type I and type II errors. 

From this result, we can also see that there is a huge class imbalance in between diabetic and non-diabetic class, which will be taken into account in the preprocessor in the next version of this analysis.

Continued efforts in these areas will contribute to the model's robustness and reliability, making it more suitable for deployment in a clinical setting.

 

## Introduction
Diabetes mellitus, commonly referred to as diabetes is a disease which impacts the body’s control of blood glucose levels {cite}`sapra_bhandari_diabetes`. It is important to note that there are different types of diabetes, although we do not explore this discrepancy in this project {cite}`sapra_bhandari_diabetes`. Diabetes is a manageable disease thanks to the discovery of insulin in 1922. Globally, 1 in 11 adults have diabetes {cite}`sapra_bhandari_diabetes`. As such, understanding the factors which are strongly related to diabetes can be important for researchers studying how to better prevent or manage the disease. In this project, we create several machine learning models to predict diabetes in a patient and evaluate the success of these models. We also explore the coefficients of a logistic regression model to better understand the factors which are associated with diabetes. 

## Methods

### Data
The dataset used in this project is a collection of the Centers for Disease Control and Prevention (CDC) diabetes health indicators collected as a response to the CDC's BRFSS2015 survey. The data were sourced from the UCI Machine Learning Repository {cite}`BurrowsEtAl` which can be found [here](https://archive.ics.uci.edu/dataset/891/cdc+diabetes+health+indicators). The file specifically used from this repository for this analysis includes 70, 692 survey responses from which 50% of the respondents recorded having either prediabetes or diabetes. Each row in the dataset represents a recorded survey response including whether or not the responded has diabetes or prediabetes, and a collection of 21 other diabetes health indicators identified by the CDC. 

### Analysis
In our efforts to determine the best model for classifying a patient with diabetes or prediabetes as opposed to no diabetes or prediabetes, we performed hyperparamter optimization on both a knn model and a decision tree model. We also explored a logistic regression model to gain insight into which features may contribute most to a classification of diabetes. All features from the original dataset were included in each model. In all cases, the data were split into training and testing datasets, with 80% of the data designated as training and 20% as testing. The data was preprocessesed such that all continuous (non-binary) variables were scaled using a scikit-learn's StandardScaler function. Model performance was tested using a 10 - fold cross validation score. Feature importance was investigated using the coefficients generated by the logistic regression algorithm. The k-nn algorithm's hyperparameter K was optimized using the F1 score as the classification metric. Python programming (Van Rossum and Drake 2009) was used for all analysis. The following Python packages were used for this analysis: Pandas {cite}`Mckinney2010data`, altair {cite}`vanderplas2018altair`, and scikit-learn {cite}`pedregosa2011scikit`.

### Results & Discussion

Before we  begin building our models for comparison, we plotted the histogram distribution of each feature in the dataset to have an insight on the features. used with 0 and 1 representing the non-diabetic class(blue) and the diabetic class(orange) respectively  {numref}`Figure {number} <feature_histogram_by_class.png>`. It is seen that for all the features, there is a significant overlapping in between the classes as most of the features are binary features. However, their max and mean values for each feature seems to be different, thus, we decided to include all the features in our model training.

```{figure} ../results/figures/feature_histogram_by_class.png
---
width: 800px
name: feature_histogram_by_class
---
Histogram distribution for each feature in the training data for both non-diabetic (0 & blue) and diabetic (1 & orange) class.
```

Using a random search, we performed hyperparameter optimization on a knn model and a decision tree model. For the knn model, we found that the optimal value for n_neighbors was {glue:text}`knn_bestparam`. For the decision tree model, we found the optimal value for max_depth to be {glue:text}`tree_bestparam`. 

We then compared the two optimized models to a dummy classifier and a logistic regression model. The optimized decision tree model out-performed all others, with a cross validation score of {glue:text} `dt_test_score`. As seen in the table below  ({numref}`Figure {number} <model_score_table>`), all of the scores were quite high for all models, even the dummy model. This is likely due to the imbalance in the target features, which should be addressed if this model were to be improved and used.

```
:figwidth: 400px
:name: "model_score_table"

Cross validation results for all optimized models, as a result of a random search hyperparamater optimization, compared to a dummy classifier.
```


Although the logistic regression model did not have the best performance compared to the other models, with a cross-validation score of {glue:text} {glue:text}`lr_test_score` its coefficients were still useful in investigating which features were weighted as more important in indicating whether a person has diabetes/prediabetes. The results, shown in the table below ({numref}`Figure {number} <coef_table>`), indicate that general health (GenHlth), is the largest predictor of diabetes/prediabetes, followed by body mass index (BMI), high blood pressure (HighBP), and age, respectively. All of these factors, unsurprisingly had a positive correlation with the presence of diabetes/prediabetes. 

```
:figwidth: 200px
:name: "coef_table"

Logistic regression coefficients by feature.
```

To enhance the future iterations of this model for potential use in medical pre-diagnosis, we propose the following recommendations:

1.	Begin by closely examining instances of {glue:text}`false_negative` misclassifications as well as {glue:text}`false_positive`, comparing them to correctly classified observations from both classes. The objective is to identify features driving misclassifications and explore potential feature engineering to improve the model's accuracy on these problematic observations.

2.	Investigate alternative classifiers to determine if enhanced predictions can be attained. A potential option is to employ a random forest classifier, which inherently considers feature interactions, in contrast to the k-NN model.

3.	Acknowledge the significant class imbalance between diabetic and non-diabetic classes and address it in the preprocessor for subsequent analyses.

4.	Enhance the model's usability in diagnosis by providing and reporting probability estimates for predictions. This approach ensures that even if misclassifications persist, clinicians with domain knowledge can gauge the model's confidence in its predictions.


## References
```{bibliography}```